# VacationPy

**Starter Code to Import Libraries and Load the Weather and Coordinates Data**

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import hvplot.pandas

from pprint import pprint
from api_keys import geoapify_key
from citipy import citipy

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,303.13,89,52,4.47,EC,1689700333
1,1,freetown,8.4840,-13.2299,299.35,82,100,3.55,SL,1689700334
2,2,rio grande,-32.0350,-52.0986,286.21,63,64,4.44,BR,1689700334
3,3,port-aux-francais,-49.3500,70.2167,278.26,84,83,17.62,TF,1689700334
4,4,edinburgh of the seven seas,-37.0676,-12.3116,287.37,96,100,11.72,SH,1689700334


In [10]:
# Convert Temperatures to Celcius
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,29.98,89,52,4.47,EC,1689700333
1,1,freetown,8.4840,-13.2299,26.20,82,100,3.55,SL,1689700334
2,2,rio grande,-32.0350,-52.0986,13.06,63,64,4.44,BR,1689700334
3,3,port-aux-francais,-49.3500,70.2167,5.11,84,83,17.62,TF,1689700334
4,4,edinburgh of the seven seas,-37.0676,-12.3116,14.22,96,100,11.72,SH,1689700334


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="Humidity",
    legend="top_left",
    hover_cols=["City"]
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values (check first to see if there are any)
check = ideal_city_df.count()
print(check)

# Display sample data
ideal_city_df

City_ID       14
City          14
Lat           14
Lng           14
Max Temp      14
Humidity      14
Cloudiness    14
Wind Speed    14
Country       14
Date          14
dtype: int64


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,94,ilgaz,40.9251,33.6259,21.87,50,0,2.51,TR,1689700374
103,103,shiguai,40.7058,110.2856,21.64,39,0,1.03,CN,1689700375
203,203,prokhladnyy,43.7574,44.0297,24.43,59,0,1.43,RU,1689700413
204,204,orchard homes,46.8633,-114.0484,25.15,38,0,1.34,US,1689700413
216,216,bandrele,-12.9067,45.1914,24.86,78,0,2.06,YT,1689700416
241,241,moses lake,47.1301,-119.2781,24.38,31,0,2.06,US,1689700423
279,279,aligudarz,33.4006,49.6949,23.22,14,0,0.24,IR,1689700434
342,342,vernon,50.2581,-119.2691,22.39,49,0,2.24,CA,1689700453
343,343,the dalles,45.5946,-121.1787,25.77,52,0,2.57,US,1689700453
434,434,nili,33.7218,66.1302,26.01,14,0,3.78,AF,1689700480


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Drop unnecessary Columns and reorder columns
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]]

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
94,ilgaz,TR,40.9251,33.6259,50,
103,shiguai,CN,40.7058,110.2856,39,
203,prokhladnyy,RU,43.7574,44.0297,59,
204,orchard homes,US,46.8633,-114.0484,38,
216,bandrele,YT,-12.9067,45.1914,78,
241,moses lake,US,47.1301,-119.2781,31,
279,aligudarz,IR,33.4006,49.6949,14,
342,vernon,CA,50.2581,-119.2691,49,
343,the dalles,US,45.5946,-121.1787,52,
434,nili,AF,33.7218,66.1302,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ilgaz - nearest hotel: Ilgaz Ögretmenevi
shiguai - nearest hotel: No hotel found
prokhladnyy - nearest hotel: Прохладная
orchard homes - nearest hotel: Broadway Inn
bandrele - nearest hotel: Les Baobabs
moses lake - nearest hotel: No hotel found
aligudarz - nearest hotel: هتل جهانگردی الیگودرز
vernon - nearest hotel: Okanagan Royal Park Inn
the dalles - nearest hotel: No hotel found
nili - nearest hotel: No hotel found
salinas - nearest hotel: The Traveler's Hotel
callao - nearest hotel: Casa Ronald
dongcun - nearest hotel: 烟台海阳方圆大酒店
portland - nearest hotel: the Hoxton


,City,Country,Lat,Lng,Humidity,Hotel Name
94,ilgaz,TR,40.9251,33.6259,50,Ilgaz Ögretmenevi
103,shiguai,CN,40.7058,110.2856,39,No hotel found
203,prokhladnyy,RU,43.7574,44.0297,59,Прохладная
204,orchard homes,US,46.8633,-114.0484,38,Broadway Inn
216,bandrele,YT,-12.9067,45.1914,78,Les Baobabs
241,moses lake,US,47.1301,-119.2781,31,No hotel found
279,aligudarz,IR,33.4006,49.6949,14,هتل جهانگردی الیگودرز
342,vernon,CA,50.2581,-119.2691,49,Okanagan Royal Park Inn
343,the dalles,US,45.5946,-121.1787,52,No hotel found
434,nili,AF,33.7218,66.1302,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",   
    legend="top_left",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)